In [1]:
import numpy as np

import time

import random

from sympy import *

# Inputs to the model

In [209]:
no_P = 3 # Number of products
no_I = 3 # Number of factories 
no_J = 3 # Number of customers
no_K = 3 # Number of ports

I = list(range(no_I)) # List of all the factories (index)
J = list(range(no_J)) # List of all the customers (index)
K = list(range(no_K)) # List of all the ports (index)

In [210]:
I_P = [sorted(random.sample(I, random.randint(1, len(I)))) for _ in range(no_P)]
J_P = [sorted(random.sample(J, random.randint(1, len(J)))) for _ in range(no_P)]
K_P = [sorted(random.sample(K, random.randint(1, len(K)))) for _ in range(no_P)]

In [211]:
assert len(I_P) == no_P, 'Set I does not have the right amount of products'
assert len(J_P) == no_P, 'Set J does not have the right amount of products'
assert len(K_P) == no_P, 'Set K does not have the right amount of products'

### Non-flatten approach

In [212]:
### All the coefficients inputs
cpik, dij, lij, aij, bij, cpi, tpi, upi, wj= [], [], [], [], [], [], [], [], []
for index in range(no_P):
    cpik.append(np.random.rand(len(I_P[index]) * len(K_P[index]))) # Cost from port to factory
    dij.append(np.random.rand(len(I_P[index]) * len(J_P[index]))) # Distance from factory to customer
    lij.append(np.random.rand(len(I_P[index]) * len(J_P[index]))) # Distance of Constant Charge
    aij.append(np.random.rand(len(I_P[index]) * len(J_P[index]))) # Amount of Constant Change
    bij.append(np.random.rand(len(I_P[index]) * len(J_P[index]))) # Rate of charge per distance
    cpi.append(np.random.rand(len(I_P[index]))) # Variable Cost
    tpi.append(np.random.rand(len(I_P[index]))) # Time taken to produce one unit
    upi.append(np.random.rand(len(I_P[index]))) # Proportion of mass loss
    wj.append(np.random.rand(len(J_P[index]))) # Demand
    
Hi = np.random.rand(len(I)) # Total Production time

In [264]:
def generate_objective_vector(no_P, dij, lij, aij, bij, cpi, cpik):
    
    ### Processing to get the objective function
    # c = cpik || (cpij + cpi)
    # We need to calculate cpij

    ### Distance Cost Function cij = max(0, b_ij(d-l)_ij) + aij
    distance_cost = lambda d, l, a, b: np.maximum(0, b*(d-l)) + a

    cpij = []

    for p in range(no_P):

        cpij.append(distance_cost(dij[p], lij[p], aij[p], bij[p]))

    ### Broadcast sum for adding together vectors with multple length
    broadcast_sum = lambda a, b: [a[i] + b[i] for i in range(len(b))]

    sum_cpij_cpi = []

    for p in range(no_P):
        sum_cpij_cpi.append(broadcast_sum(np.split(cpij[p], len(cpi[p])), cpi[p]))

    left = np.concatenate(cpik) # This is the cpik
    right = np.concatenate([np.concatenate(elem) for elem in sum_cpij_cpi]) # This is the cpi

    ### Create one long vector c
    c = np.concatenate((left, right))
    
    return c

### Constraints

#### 1. Demand Satisfaction

In [220]:
def create_skeleton(cpik, cpij):
    
    # This just replace all the elemnts with 0, whether to reshape or concatenate will be decided later onz
    left = [np.zeros(elem.shape) for elem in cpik]
    right = [np.zeros(elem.shape) for elem in cpij]
    
    return left, right

In [221]:
len_IP = np.array([len(elem) for elem in I_P])
len_JP = np.array([len(elem) for elem in J_P])
len_KP = np.array([len(elem) for elem in K_P])

In [222]:
left, right = create_skeleton(cpik, cpij)

reshape_left = []

for index, product in enumerate(left):
    reshape_left.append(product.reshape(  len_IP[index], len_KP[index] ) ) #(|I_p|, |K_p|)

reshape_right = []

for index, product in enumerate(right):
    reshape_right.append(product.reshape(  len_IP[index], len_JP[index]) ) #(|I_p|, |J_p|)

In [223]:
copy_list = lambda mat_list: [np.copy(elem) for elem in mat_list]

In [224]:
### Maybe in the future optimize this?
zeroed_left = np.zeros((np.sum(len_JP), np.sum(len_IP*len_KP)))

ones_right = []

for p in range(no_P):
    
    ones_right_P = []
    for j_index in range(len(J_P[p])):
        row = copy_list(reshape_right) # Copy the empty matrix

        ## Check that the slice is equal to the number of factories
        assert len(row[p][:, j_index]) == len(I_P[p])

        row[p][:, j_index] = -1 # Negative to minimize
        
        matrix_row = np.concatenate([np.concatenate(elem) for elem in row])

        ones_right_P.append(matrix_row)
        
    ones_right.append(ones_right_P)
    
### This still need the zero block
ones_right = np.concatenate(ones_right)
demand_matrix = np.concatenate((zeroed_left, ones_right), axis = 1) 
wj_final = -np.concatenate(wj) # Negative to minimize

assert demand_matrix.shape == (len(wj_final), len(c)), 'Demand Matrix not of the Correct shape'

#### 2. Factory Capacity

In [225]:
def get_factory_product_list(I, I_P):

    # For each factory which product does it produce, and the position of the factory in the I_P list
    factory_product_list = []

    for factory in I:

        factory_product = []
        for product in I_P:
            if factory in product:
                factory_product.append(product.index(factory))
            else:
                factory_product.append(None)

        factory_product_list.append(factory_product)
        
    return factory_product_list

In [226]:
factory_product_list = get_factory_product_list(I, I_P)

In [227]:
zeroed_left = np.zeros((len(I), np.sum(len_IP*len_KP)))

ones_right = []

for factory_index_list in factory_product_list:
    row = copy_list(reshape_right)

    for p, i in enumerate(factory_index_list):
        
        if i is None:
            pass
        else:
            row[p][i] = tpi[p][i]
            
    matrix_row = np.concatenate([np.concatenate(elem) for elem in row])
    ones_right.append(matrix_row)
    
capacity_matrix = np.hstack((zeroed_left, np.vstack(ones_right)))

assert capacity_matrix.shape == (len(I), len(c)), 'Demand Matrix not of the Correct shape'

#### 3. Sufficient Supply

$$V_{p, i}^{\text{out}} - \mu_{p, i}V_{p, i}^{\text{in}} \leq 0 $$

$$\sum_{k \in K_p} v_{ik} - \mu_{p, i}\sum_{j \in J_p} v_{ij}$$

In [228]:
def get_port_product_list(K, K_P):

    # For each port which product comes from it, and the position of the port in the K_P list
    port_product_list = []

    for port in K:

        port_product = []
        for product in K_P:
            if port in product:
                port_product.append(product.index(port))
            else:
                port_product.append(None)

        port_product_list.append(port_product)
        
    return port_product_list

port_product_list = get_port_product_list(K, K_P)

In [229]:
matrix_row = []
for p in range(no_P):
    for i in range(len_IP[p]):
        left_row = copy_list(reshape_left)
        right_row = copy_list(reshape_right)
        
        left_row[p][i] = -upi[p][i]
        right_row[p][i] = 1
        
        left_row = [np.concatenate(elem) for elem in left_row]
        right_row = [np.concatenate(elem) for elem in right_row]
        matrix_row.append(np.concatenate(left_row + right_row))
        
supply_matrix = np.vstack(matrix_row)

assert supply_matrix.shape == (sum(len_IP), len(c)), 'Supply Matrix not of the Correct shape'

In [230]:
A = np.vstack((demand_matrix, capacity_matrix, supply_matrix))

b = np.concatenate([wj_final, Hi, np.zeros((sum(len_IP)))])

In [246]:
A = np.vstack((demand_matrix, capacity_matrix))
b = np.concatenate([wj_final, Hi])

In [247]:
from scipy.optimize import linprog

In [248]:
linprog?

In [268]:
b

array([-0.84766375, -0.88189947, -0.27096431, -0.16578414, -0.92606573,
        0.33999578,  0.8683442 ,  0.92901821])

In [262]:
linprog(c, A_ub = A[:-1], b_ub = b[:-1], options = {'maxiter': 100})

     con: array([], dtype=float64)
     fun: 5.162826914338323
 message: 'The algorithm terminated successfully and determined that the problem is infeasible.'
     nit: 5
   slack: array([ 0.09909145,  0.98496101,  0.00679311,  1.02228076,  0.20124197,
       -0.58471213, -0.02283008])
  status: 2
 success: False
       x: array([0.        , 0.        , 0.        , 0.        , 0.9467552 ,
       1.86686048, 0.27775741, 1.1880649 , 1.1273077 ])